In [13]:
import boto3

In [14]:
%%writefile Dockerfile

FROM python:3.9-slim-buster

COPY requirements.txt /opt/ml/requirements.txt
COPY code/preprocess.py /opt/ml/code/preprocess.py
COPY code/train.py /opt/ml/code/train.py

RUN pip3 install -r /opt/ml/requirements.txt

ENV SAGEMAKER_PROGRAM preprocess.py
ENV SAGEMAKER_PROGRAM train.py
ENV PYTHONUNBUFFERED=TRUE

ENTRYPOINT ["python3", "/opt/ml/code/train.py"]

Overwriting Dockerfile


In [15]:
account_id = boto3.client("sts").get_caller_identity().get("Account")
region = boto3.Session().region_name
ecr_repository = "prophet_training"
tag = ":latest"
processing_repository_uri = "{}.dkr.ecr.{}.amazonaws.com/{}".format(
    account_id, region, ecr_repository + tag
)

In [16]:
# Create ECR repository and push docker image
!docker build -t $ecr_repository .

Sending build context to Docker daemon    278kB
Step 1/9 : FROM python:3.9-slim-buster
 ---> c84dbfe3b8de
Step 2/9 : COPY requirements.txt /opt/ml/requirements.txt
 ---> Using cache
 ---> 7f1423ae1d9f
Step 3/9 : COPY code/preprocess.py /opt/ml/code/preprocess.py
 ---> Using cache
 ---> eddc8197cc77
Step 4/9 : COPY code/train.py /opt/ml/code/train.py
 ---> 9c56066459e4
Step 5/9 : RUN pip3 install -r /opt/ml/requirements.txt
 ---> Running in 664f641e2c38
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 90.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 60.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 49.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 27.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 k

In [17]:
!aws ecr get-login-password --region {region} | docker login --username AWS --password-stdin {account_id}.dkr.ecr.{region}.amazonaws.com

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded


In [18]:
!aws ecr create-repository --repository-name $ecr_repository

{
    "repository": {
        "repositoryArn": "arn:aws:ecr:ap-south-1:654654222480:repository/prophet_training",
        "registryId": "654654222480",
        "repositoryName": "prophet_training",
        "repositoryUri": "654654222480.dkr.ecr.ap-south-1.amazonaws.com/prophet_training",
        "createdAt": 1723970791.99,
        "imageTagMutability": "MUTABLE",
        "imageScanningConfiguration": {
            "scanOnPush": false
        },
        "encryptionConfiguration": {
            "encryptionType": "AES256"
        }
    }
}


In [19]:
# aws ecr describe-repositories --repository-names "${algorithm_name}" > /dev/null 2>&1
# if [ $? -ne 0 ]
# then
# aws ecr create-repository --repository-name "${algorithm_name}" > /dev/null
# fi

In [20]:
!docker tag {ecr_repository + tag} $processing_repository_uri

In [21]:
!docker push $processing_repository_uri

The push refers to repository [654654222480.dkr.ecr.ap-south-1.amazonaws.com/prophet_training]

ba568792: Preparing 
3cd29314: Preparing 
3b9a5f4f: Preparing 
087666fd: Preparing 
a27560c1: Preparing 
037e08b3: Preparing 
eede8d6e: Preparing 
55769c5e: Preparing 
ba568792: Pushed   657.9MB/627.5MBPushed latest: digest: sha256:04f21d6456d0aaf1f764d3312176d1f3cd7bb424b1107db366910910738c4403 size: 2204
